#### HOTELS ATTRIBUTES EXTRACTION PROCESS

This notebook was used to execute manually the scripts to scrape all of the previously scrapped hotels attributes.

From the scrapping.py file we import scrape_hotels_attributes, this function takes a list of urls and returns a dataframe with the following hotel data:

- attributes
- scores

The data was scrapped from the [Booking.com](https://www.booking.com/index.es.html?label=gen173nr-1BCAEoggI46AdIM1gEaAyIAQGYAQq4ARfIAQzYAQHoAQGIAgGoAgO4At_puqsGwAIB0gIkOTkxOTFjMmEtNTEzYS00NDQyLTgxNmMtMWJmMzMyYzVjNjJj2AIF4AIB&sid=1a3dea14733e69dc474ff74d1fad21e9&keep_landing=1&sb_price_type=total&) web, from a page like this: [boooking/hotels/newyork/hotel.com](https://www.booking.com/hotel/us/west-side-ymca.es.html?aid=304142&label=gen173nr-1FCAEoggI46AdIM1gEaAyIAQGYAQq4ARfIAQzYAQHoAQH4AQyIAgGoAgO4Arj9uqsGwAIB0gIkMzNmMDQ2ZDMtOTkwYy00ZWVmLTgwZDYtYzhjMWUxZmFjM2Yz2AIG4AIB&sid=75f60e0ba4b489f2fe2baf48281729c2&dest_id=20088325;dest_type=city;dist=0;group_adults=2;group_children=0;hapos=4;hpos=4;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;srepoch=1701756621;srpvid=525f2b61586300dd;type=total;ucfs=1&#hotelTmpl).

In [1]:
import os
import sys
import scrapping
import pandas as pd

usa_cities = pd.read_csv("../files/data/usa_cities.csv")
usa_states = pd.read_csv("../files/data/usa_states.csv")
usa_hotels = pd.read_csv("../files/data/usa_hotels.csv", index_col=0)

California

In [2]:
california_hotels = usa_hotels[usa_hotels["state"] == "California"]
california_hotels_url_list = california_hotels["reviews_url"].tolist()
california_hotels_ids_list = california_hotels["hotel_id"].tolist()

In [3]:
california_hotels_attributes_1_3 = scrapping.scrape_hotels_attributes(california_hotels_url_list[:500], california_hotels_ids_list[:500])
california_hotels_attributes_1_3.to_csv("../files/data/booking/attributes/california_hotels_attributes_1_3.csv")
california_hotels_attributes_1_3

1/500


c:\Users\david\Documents\GitHub\PF_DS_YELP\scrapping\scrapping.py:273: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hotels_attributes = pd.concat([hotels_attributes, attributes], ignore_index=True, sort=False)


2/500
3/500
4/500
5/500
6/500
7/500
8/500
9/500
10/500
11/500
12/500
Error StaleElementReferenceException obteniendo scores para el hotel con ID 351
13/500
14/500
15/500
16/500
17/500
18/500
19/500
20/500
Error StaleElementReferenceException obteniendo scores para el hotel con ID 592
21/500
22/500
23/500
24/500
25/500
26/500
27/500
28/500
29/500
30/500
31/500
32/500
33/500
34/500
35/500
36/500
37/500
38/500
39/500
40/500
41/500
42/500
43/500
44/500
45/500
46/500
47/500
48/500
49/500
50/500
51/500
52/500
53/500
54/500
55/500
56/500
57/500
58/500
59/500
60/500
61/500
62/500
63/500
64/500
65/500
66/500
67/500
68/500
Error StaleElementReferenceException obteniendo scores para el hotel con ID 2100
69/500
70/500
71/500
Error StaleElementReferenceException obteniendo scores para el hotel con ID 2118
72/500
73/500
74/500
75/500
76/500
Error StaleElementReferenceException obteniendo scores para el hotel con ID 2228
77/500
Error StaleElementReferenceException obteniendo scores para el hotel con 

,hotel_id,direction,description,attributes,latitude,longitude,scores
0,16,"24232 Tahoe Court, Laguna Niguel, CA 92677, Es...","Breathtaking views, close to the ocean and can...","[Habitaciones familiares, ¿Hay spa?, Parking g...",33.552129,-117.706753,[]
1,27,"220 20th Street, Newport Beach, CA 92663, Esta...","Surf Casita está en Newport Beach, a 3 min a p...","[Habitaciones familiares, Parking gratis, Vete...",33.609155,-117.926502,[]
2,35,"Pacific Beach, San Diego, 92109-5890, Estados ...",Upscale Beach Rowhome with Rooftop and Bay Vie...,"[Parking en el alojamiento, ¿Hay piscina?, ¿Ha...",32.790681,-117.240283,[]
3,39,"Julian Avenue 2054, San Diego, 92113, Estados ...",Victorian Retreat in Barrio Logan 1mi to Downt...,"[¿Hay spa?, ¿Hay piscina?, Parking gratis, ¿Có...",32.701546,-117.139944,[]
4,44,"4050 Albatross Street, Unit 4048, Hillcrest, S...",4048 Stylish condo in Hillcrest Mission Hills ...,"[Habitaciones familiares, ¿Hay spa?, ¿Hay pisc...",32.746639,-117.166122,[]
...,...,...,...,...,...,...,...
495,15383,"Washington Barbot 59, 70000 Colonia del Sacram...",A sólo 1 calle del pintoresco circuito históri...,"[Bar, Puerta de la Ciudadela de Colonia\n200 m...",-34.472865,-57.848519,"[{'Personal': '9.6'}, {'Instalaciones y servic..."
496,15384,"Avenida Roosevelt, 381, 70000 Colonia del Sacr...",Este hotel se encuentra en Colonia del Sacrame...,"[Habitaciones familiares, Bar, Servicio de hab...",-34.467040,-57.837197,"[{'Personal': '9.2'}, {'Instalaciones y servic..."
497,15390,"8312 Espresso Drive, Bakersfield, 93312, Estad...",Residence Inn by Marriott Bakersfield West se ...,"[Desayuno, Habitaciones familiares, Parking gr...",35.388858,-119.094794,"[{'Personal': '9.2'}, {'Instalaciones y servic..."
498,15396,"325 Bristol Street, Costa Mesa, CA 92626, Esta...",El Ayres Hotel & Suites Costa Mesa/Newport Bea...,"[Habitaciones familiares, Bar, Servicio de hab...",33.665531,-117.881957,"[{'Personal': '9.0'}, {'Instalaciones y servic..."


In [ ]:
california_hotels_attributes_2_3 = scrapping.scrape_hotels_attributes(california_hotels_url_list[500:1000], california_hotels_ids_list[500:1000])
california_hotels_attributes_2_3.to_csv("../files/data/booking/attributes/california_hotels_attributes_2_3.csv")
california_hotels_attributes_2_3

1/500


c:\Users\david\Documents\GitHub\PF_DS_YELP\scrapping\scrapping.py:273: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hotels_attributes = pd.concat([hotels_attributes, attributes], ignore_index=True, sort=False)


2/500
3/500
4/500
5/500
6/500
7/500
8/500
9/500
10/500
Error StaleElementReferenceException obteniendo scores para el hotel con ID 15429
11/500
12/500
13/500
14/500
Error StaleElementReferenceException obteniendo scores para el hotel con ID 15458
15/500
16/500
17/500
18/500
19/500
20/500
21/500
22/500
Error StaleElementReferenceException obteniendo scores para el hotel con ID 15531
23/500
24/500
25/500
26/500
27/500
28/500
29/500
30/500
Error StaleElementReferenceException obteniendo scores para el hotel con ID 15624
31/500
32/500
33/500
34/500
35/500
36/500
37/500
38/500
39/500
40/500
41/500
42/500
43/500
Error StaleElementReferenceException obteniendo scores para el hotel con ID 15714
44/500
45/500
46/500
47/500
48/500
49/500
50/500
51/500
52/500
53/500
54/500
55/500
56/500
57/500
58/500
59/500
60/500
61/500
62/500
63/500
64/500
65/500
66/500
67/500
68/500
69/500
70/500
71/500
72/500
73/500
74/500
75/500
76/500
77/500
78/500
79/500
80/500
Error StaleElementReferenceException obtenien

,hotel_id,direction,description,attributes,latitude,longitude,scores
0,15399,"10850 Lindbrook Drive, Westwood, Los Ángeles, ...",El Plaza La Reina está situado en Westwood Vil...,"[Recepción 24 horas, Habitaciones familiares, ...",34.059996,-118.442316,"[{'Personal': '9.0'}, {'Instalaciones y servic..."
1,15400,"28941 Los Alisos Boulevard, Mission Viejo, CA ...",El Ayres Suites Mission Viejo ofrece una pisci...,"[Parking gratis, Gimnasio, Spa y centro de bie...",33.653829,-117.627143,"[{'Personal': '9.1'}, {'Instalaciones y servic..."
2,15401,"460 North Pacific Coast Highway, Laguna Beach,...",Este motel se encuentra a 10 minutos a pie del...,"[Parking gratis, Tetera/cafetera en todas las ...",33.545069,-117.791293,"[{'Personal': '9.5'}, {'Instalaciones y servic..."
3,15402,"28951 Los Alisos Boulevard, Mission Viejo, CA ...",El Ayres Hotel & Spa Mission Viejo es un hotel...,"[Gimnasio, Restaurante, WiFi gratis, Buen desa...",33.653896,-117.626898,"[{'Personal': '9.0'}, {'Instalaciones y servic..."
4,15408,"Calle Luisa, 10108 San José, Costa Rica","7 De Marzo Sabana está en San José, cerca de P...","[Parking gratis, Parque República del Salvador...",9.939215,-84.101985,"[{'Personal': '9.1'}, {'Instalaciones y servic..."
...,...,...,...,...,...,...,...
495,26576,"4240 La Jolla Village Drive, La Jolla, San Die...","Este hotel está ubicado en La Jolla, a 5 minut...","[Recepción 24 horas, Gimnasio, WiFi, Servicio ...",32.873055,-117.215936,"[{'Personal': '8.6'}, {'Instalaciones y servic..."
496,26600,"600 South Spring Street, Centro de Los Ángeles...","Luxury spacious downtown loft, que cuenta con ...","[Gimnasio, Aire acondicionado, Ascensor, WiFi ...",34.045570,-118.250926,"[{'Personal': '8.2'}, {'Instalaciones y servic..."
497,26602,"1132 Prospect Street, La Jolla, San Diego, CA ...",Este hotel de estilo mediterráneo está situado...,"[Gimnasio, Restaurante, Tetera/cafetera en tod...",32.849034,-117.274048,"[{'Personal': '8.4'}, {'Instalaciones y servic..."
498,26603,"715 S Kingsley Dr, Los Ángeles, CA 90005, Esta...",LA Cozy 1 bedroom FREE parking se encuentra en...,"[Parking gratis, Aire acondicionado, Nevera, W...",34.059163,-118.303250,"[{'Personal': '7.8'}, {'Instalaciones y servic..."


In [ ]:
california_hotels_attributes_3_3 = scrapping.scrape_hotels_attributes(california_hotels_url_list[1000:], california_hotels_ids_list[1000:])
california_hotels_attributes_3_3.to_csv("../files/data/booking/attributes/california_hotels_attributes_3_3.csv")
california_hotels_attributes_3_3

1/556


c:\Users\david\Documents\GitHub\PF_DS_YELP\scrapping\scrapping.py:273: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hotels_attributes = pd.concat([hotels_attributes, attributes], ignore_index=True, sort=False)


2/556
3/556
4/556
5/556
6/556
7/556
8/556
9/556
10/556
11/556
12/556
13/556
14/556
15/556
16/556
17/556
18/556
19/556
20/556
21/556
22/556
23/556
24/556
25/556
26/556
27/556
28/556
29/556
30/556
31/556
32/556
33/556
34/556
35/556
36/556
37/556
38/556
39/556
40/556
41/556
42/556
43/556
44/556
45/556
46/556
47/556
48/556
49/556
50/556
51/556
52/556
53/556
54/556
55/556
56/556
57/556
58/556
59/556
60/556
61/556
62/556
63/556
64/556
65/556
66/556
67/556
68/556
69/556
70/556
71/556
72/556
73/556
74/556
75/556
76/556
77/556
Error StaleElementReferenceException obteniendo scores para el hotel con ID 28504
78/556
79/556
80/556
81/556
82/556
83/556
84/556
85/556
86/556
87/556
88/556
89/556
90/556
91/556
92/556
93/556
94/556
95/556
96/556
97/556
98/556
99/556
100/556
101/556
102/556
103/556
104/556
105/556
106/556
107/556
108/556
109/556
110/556
111/556
112/556
113/556
114/556
115/556
116/556
117/556
118/556
119/556
120/556
121/556
122/556
123/556
124/556
125/556
126/556
127/556
128/556
129/556


,hotel_id,direction,description,attributes,latitude,longitude,scores
0,26614,"1928 18th Street Apt #E, Santa Mónica, Los Áng...",Spacious Two Bedroom in Santa Monica está en L...,"[¿Hay piscina?, WiFi gratis, ¿Hay spa?, ¿Se si...",34.019617,-118.473552,[]
1,26629,"127 Ellis Street, Union Square, San Francisco,...",Hotel Abri Union Square es un alojamiento con ...,"[Parking privado, WiFi gratis, Adaptado person...",37.785364,-122.408369,"[{'Personal': '8.4'}, {'Instalaciones y servic..."
2,26645,"242 Powell Street, Union Square, San Francisco...",El Stratford Hotel tiene una buena ubicación e...,"[Recepción 24 horas, WiFi gratis, Adaptado per...",37.787055,-122.408073,"[{'Personal': '8.2'}, {'Instalaciones y servic..."
3,26693,"Union Square, San Francisco, CA 94109, Estados...",Kasa La Monarca San Francisco está muy bien si...,"[Tenderloin Recreation Center\n500 m, WiFi gra...",37.788489,-122.412215,"[{'Personal': '7.6'}, {'Instalaciones y servic..."
4,26718,"777 North Vine Street, Hollywood, Los Ángeles,...",El Rodeway Inn near Melrose Ave se encuentra a...,"[Recepción 24 horas, Parking privado, WiFi gra...",34.085145,-118.326766,"[{'Personal': '7.7'}, {'Instalaciones y servic..."
...,...,...,...,...,...,...,...
551,36697,"2044 N Sycamore Ave , Hollywood, Los Ángeles, ...",Sycamore View Villa I se encuentra en el barri...,"[WiFi gratis, ¿Hay piscina?, ¿Hay spa?, ¿Se si...",34.107774,-118.342226,[]
552,36703,"3 Paseo Vespertino, Rancho Santa Margarita, CA...",King-30min to Airport-Self Check In-Parking-36...,"[¿Hay piscina?, WiFi gratis, ¿Hay spa?, ¿Se si...",33.640097,-117.597847,[]
553,36710,"25 Pacifica, Irvine, CA 92618, Estados Unidos",Irvine Spectrum/2 Bedrooms/2 Bathrooms/Apartme...,"[Piscina al aire libre, ¿Está abierta la pisci...",33.651016,-117.750711,"[{'Personal': '10'}, {'Instalaciones y servici..."
554,36713,"31861 Coast Highway, Laguna Beach, CA 92651, E...",OCEAN FRONT Beach HOUSE! Private Stairs to SAN...,"[WiFi gratis, ¿Hay piscina?, Habitaciones fami...",33.499589,-117.743265,[]


In [ ]:
attributes_3_3 = pd.read_csv("../files/data/booking/attributes/california_hotels_attributes_3_3.csv", index_col=0)

In [8]:
attributes_1_3 = pd.read_csv("../files/data/booking/attributes/california_hotels_attributes_1_3.csv", index_col=0)
attributes_2_3 = pd.read_csv("../files/data/booking/attributes/california_hotels_attributes_2_3.csv", index_col=0)
attributes_3_3 = pd.read_csv("../files/data/booking/attributes/california_hotels_attributes_3_3.csv", index_col=0)
california_hotels_attributes = pd.concat([attributes_1_3, attributes_2_3, attributes_3_3], ignore_index=True)
os.remove("../files/data/booking/attributes/california_hotels_attributes_1_3.csv")
os.remove("../files/data/booking/attributes/california_hotels_attributes_2_3.csv")
os.remove("../files/data/booking/attributes/california_hotels_attributes_3_3.csv")
california_hotels_attributes.to_csv("../files/data/booking/attributes/california_hotels_attributes.csv")

,Unnamed: 0,hotel_id,direction,description,attributes,latitude,longitude,scores
0,0,16,"24232 Tahoe Court, Laguna Niguel, CA 92677, Es...","Breathtaking views, close to the ocean and can...","['Habitaciones familiares', '¿Hay spa?', 'Park...",33.552129,-117.706753,[]
1,1,27,"220 20th Street, Newport Beach, CA 92663, Esta...","Surf Casita está en Newport Beach, a 3 min a p...","['Habitaciones familiares', 'Parking gratis', ...",33.609155,-117.926502,[]
2,2,35,"Pacific Beach, San Diego, 92109-5890, Estados ...",Upscale Beach Rowhome with Rooftop and Bay Vie...,"['Parking en el alojamiento', '¿Hay piscina?',...",32.790681,-117.240283,[]
3,3,39,"Julian Avenue 2054, San Diego, 92113, Estados ...",Victorian Retreat in Barrio Logan 1mi to Downt...,"['¿Hay spa?', '¿Hay piscina?', 'Parking gratis...",32.701546,-117.139944,[]
4,4,44,"4050 Albatross Street, Unit 4048, Hillcrest, S...",4048 Stylish condo in Hillcrest Mission Hills ...,"['Habitaciones familiares', '¿Hay spa?', '¿Hay...",32.746639,-117.166122,[]
...,...,...,...,...,...,...,...,...
1551,551,36697,"2044 N Sycamore Ave , Hollywood, Los Ángeles, ...",Sycamore View Villa I se encuentra en el barri...,"['WiFi gratis', '¿Hay piscina?', '¿Hay spa?', ...",34.107774,-118.342226,[]
1552,552,36703,"3 Paseo Vespertino, Rancho Santa Margarita, CA...",King-30min to Airport-Self Check In-Parking-36...,"['¿Hay piscina?', 'WiFi gratis', '¿Hay spa?', ...",33.640097,-117.597847,[]
1553,553,36710,"25 Pacifica, Irvine, CA 92618, Estados Unidos",Irvine Spectrum/2 Bedrooms/2 Bathrooms/Apartme...,"['Piscina al aire libre', '¿Está abierta la pi...",33.651016,-117.750711,"[{'Personal': '10'}, {'Instalaciones y servici..."
1554,554,36713,"31861 Coast Highway, Laguna Beach, CA 92651, E...",OCEAN FRONT Beach HOUSE! Private Stairs to SAN...,"['WiFi gratis', '¿Hay piscina?', 'Habitaciones...",33.499589,-117.743265,[]
